# Exploratory Data Analysis and Modeling for static models
**This notebook serves as the main analysis module in this project**

Since the time series analysis is not the first priority, I will first establish static classification models such as tree models and non-time dependent survival models. I will look at the history of the patients before confirmation of prediabetes and assume that the state of the patients was determined at that moment.


The notebook has the following sections:
1. Data Processing and Tables Joining
2. Data Overview
3. Missing Data
4. Preprocessing
5. Feature Selection
6. Resample Data
7. Machine Learning Model
8. Deep Learning Model

For clarity: we name all the pre-diabetes patients with prefix pre,diabetes patients with prefix diab and the patients progressed from pre-diabetes to diabetes with prefix pre2Diab.

In [ ]:
#import libraries
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pickle

import util.cleaning_tools as tools


import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,\
precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier, Lasso
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier

from random import sample
import time
import warnings
import json
import scipy.stats as stats
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

In [ ]:
# hyepr-parameter
TIME_SPEC = 2
CUT_OFF = '{year}-12-31'.format(year=2019-TIME_SPEC)
log = {}
OUT_PATH = f"../output/george/spec-{TIME_SPEC}year/"
name_dict = None

#define indicators id
term_id = None
  

## 1. Data processing and tables joining
This section includes steps:
1. Read the patients and test data.
2. Select the target tests(indicators).
3. Join the tables.
4. Only inlcude the tests within six month before diagnosis of pre-diabetes.
5. Aggregate

### 1.1 Read the patients and test data
Read the data from the output, here the patients tables contains the detailed information about the patients.

In [ ]:
# patients data
patients = pd.read_csv(f'../tables/output/diab_patient-{TIME_SPEC}year.csv', index_col=0)

# define the file path and tables path for file reading
file_path = r'../DATAFILE'
tid_to_eid_path = r'iams_entity_concept'
labresult_cps_path = 'lis_cps_result_data'
labresult_hms_path = 'lis_hms_result_data'

# read the fragment files and concat them
usecols = ["pseudo_patient_key", "reference_dtm", "diff_in_hour_reference_dtm", "result_str", "entity_id", "si_unit", "si_numeric"]
labresult_cps = tools.fileReader(file_path, labresult_cps_path, usecols=usecols)
labresult_hms = tools.fileReader(file_path, labresult_hms_path, usecols=usecols)
tid_desc = tools.fileReader(r'../DATAFILE', "iams_concept") #term id description
tid_to_eid = tools.fileReader(file_path, tid_to_eid_path)

# the datafield of cps and hms are the same, so we can concate them.
labresult = pd.concat([labresult_cps, labresult_hms])

# # district information
# cols = ["pseudo_patient_key", "district_board"]
# district_mapp = tools.fileReader(file_path, "map_pmi_district")
# emg = tools.fileReader(file_path, "aeis_case_data", usecols = cols)
# oaa = tools.fileReader(file_path, "opas_case_data", usecols = cols)
# ipt = tools.fileReader(file_path, "ipas_case_data1", usecols = cols)
# district_record = pd.concat([emg, oaa, ipt])
# district_record.replace(r'""', np.nan, inplace=True)
# district_record.dropna(how='any', inplace=True)
# patient_district = district_record.groupby("pseudo_patient_key").apply(lambda x : x.iloc[0])
# patient_district.reset_index(drop=True, inplace=True)

### 1.2 Select the target tests(indicators)

In [ ]:
target_id = pd.Series(term_id).rename('term_id')
target_tid_mapping = pd.merge(target_id,tid_to_eid,how='left',on='term_id')
labresult_filtered = pd.merge(labresult,target_tid_mapping,how='inner',on='entity_id')
#replace the null value with np.nan
labresult_filtered.replace(r'""', np.nan, inplace=True)

### 1.3 Join the tables

Left join the patients information(id, pre hour and diab hour) with lab-result by the patient key to match all the tests for each patient.

In [ ]:
#left join the patients tables and test tables 
patients_test = pd.merge(left=patients[["pseudo_patient_key", "pre_diff_hour", "diab_diff_hour"]], 
                         right=labresult_filtered[["pseudo_patient_key", "term_id", "reference_dtm", "diff_in_hour_reference_dtm", "si_unit","si_numeric" ]], 
                         on="pseudo_patient_key", 
                         how="left")
#rename
patients_test.rename(columns = {"diff_in_hour_reference_dtm": "test_diff_hour", "reference_dtm": "test_dtm"}, inplace=True)

### 1.4 Aggregate

Aggregate each tests with its mean value and pivot table so that each test stands for a single feaures.

Set time limit for including features: the tests must lies within **6 months** before the prediabetes diagnosis and then use the *mean value* for each test in this period.

In [ ]:
# select the observations that the test is within six month before the 
# prediabetes diagnosis to three months after
# patients_test_filtered = patients_test.query("test_diff_hour > (pre_diff_hour - 6 * 30 * 24) and test_diff_hour < (pre_diff_hour + 3 * 60 * 24)")
patients_test_filtered = patients_test.query("test_diff_hour <= pre_diff_hour and test_diff_hour > (pre_diff_hour - 6 * 30 * 24)")
# drop the observations that the test is later than diagnosis of diabetes
# patients_test_filtered = patients_test_filtered.query("diab_diff_hour.isnull() or test_diff_hour < diab_diff_hour", engine="python")

# only keep the patient id, term_id and test results
patients_test_filtered = patients_test_filtered[["pseudo_patient_key", "term_id", "test_diff_hour", "si_numeric"]]
# casting type for si_numeric
patients_test_filtered["si_numeric"] = patients_test_filtered["si_numeric"].astype("float")

In [ ]:
# group the data by patients and termid and aggregate the test result with latest test value
patients_test_group = patients_test_filtered\
    .sort_values(by=["pseudo_patient_key", "test_diff_hour"])\
    .groupby(["pseudo_patient_key", "term_id"], as_index=False)\
    .agg({"si_numeric":"mean"})

In [ ]:
# table pivot
patients_features_pivoted = patients_test_group.pivot_table(index="pseudo_patient_key", columns="term_id", values="si_numeric")

In [ ]:
# reset index
patients_features_pivoted = patients_features_pivoted.reset_index()

In [ ]:
patients_features_pivoted

Rename the tests

In [ ]:
# rename all the tests out of interest
patients_features_pivoted.rename(columns=name_dict, inplace=True)

In [ ]:
patients_features_pivoted

Join the dmcs variables with test results

In [ ]:
# data cleaning
dataset =  pd.merge(left=patients_features_pivoted, right=patients, how="inner", on="pseudo_patient_key") # join the with the patient information
dataset = dataset.query("HBA1C < 6.4 or HBA1C.isnull()", engine='python')
dataset = dataset.query("cholesLDL_1 > 0 or cholesLDL_1.isnull()", engine='python')

In [ ]:
# write to disk
# dataset.to_csv(f"../tables/output/dataset-{TIME_SPEC}year")
dataset = pd.read_csv(f"../tables/output/dataset-{TIME_SPEC}year", index_col=0)

## 2. Data Overview

This section report the overall statistic of the data which will inlcude following topics:
1. Mean, standard deviance and portion of each class for each test features.
2. F-test for each test features.
3. Correlation among these significant features.

In [ ]:
dataset = pd.read_csv(f"../tables/output/dataset-{TIME_SPEC}year", index_col=0)
# dataset.drop(["Unnamed: 0.1"], axis=1, inplace=True)

## Test unit

In [ ]:
# test progression period time
assert (dataset.diab_diff_hour - dataset.pre_diff_hour - dataset.prog_pd).sum() == 0

In [ ]:
# test progression class

# null progression period should be class 0
assert (dataset.query("prog_pd.isnull()", engine='python').cls != 0).sum() == 0

# progression period greater than time spectrum should be class 0
h = TIME_SPEC * 365.25 * 24
assert (dataset.query("prog_pd >= {hour}".format(hour=h)).cls != 0).sum() == 0

# progression period less than time sepctrum should be class 1
assert (dataset.query("prog_pd < {hour}".format(hour=h)).cls != 1).sum() == 0

### 2.1 Chi Square test and T test for each variable

In [ ]:
mean = []
std = []
missing_rate = []
pvalue = []
indicators = ["pre_age", "sex"] + list(name_dict.values())
pos_mean = []
neg_mean = []
pos_std = []
neg_std = []

for ind in indicators:
    if ind == "sex":
        temp = dataset[['sex', 'cls']]
        male_0 = temp.query("sex == 'M' and cls == 0").count()
        male_1 = temp.query("sex == 'M' and cls == 1").count()
        female_0 = temp.query("sex == 'F' and cls == 0").count()
        female_1 = temp.query("sex == 'F' and cls == 1").count()
        result = stats.chi2_contingency([[male_0, female_0], [male_1, female_1]])
        p_value = result[1]
        pvalue.append(p_value)
        mean.append(np.nan)
        std.append(np.nan)
        pos_mean.append(np.nan)
        neg_mean.append(np.nan)
        pos_std.append(np.nan)
        neg_std.append(np.nan)
    else:
        temp = dataset[[ind, "cls"]]
        temp = temp[temp[ind].notnull()]
        t0 = temp.query("cls == 0")[ind]
        t1 = temp.query("cls == 1")[ind]
        result = stats.ttest_ind(t0, t1)
        pvalue.append(result.pvalue)
        mean.append(temp[ind].mean())
        std.append(temp[ind].std())
        pos_mean.append(t1.mean())
        neg_mean.append(t0.mean())
        pos_std.append(t1.std())
        neg_std.append(t0.std())

In [ ]:
stt = pd.DataFrame({
    "feauture": indicators,
    "mean": mean,
    "standard deviance": std,
    "neg_mean": neg_mean,
    "neg_std": neg_std,
    "pos_mean": pos_mean,
    "pos_std": pos_std,
    "p-value": pvalue,
})

### 2.2 Statistics of each feature

Check the overall statistic.

In [ ]:
# sex
sex_stt = dataset.groupby(["cls", "sex"])["pseudo_patient_key"].count().reset_index()
sex_stt["portion"] = sex_stt["pseudo_patient_key"] / sex_stt["pseudo_patient_key"].sum()

In [ ]:
# chi-square for gender

print("p-value for sex is: ", p_value)

## 3. Missing value
In this section, 
1. explore the missing data of each feature.
2. exlcude invalid features

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression, LogisticRegression

### 3.1 Explore the missing data of each feature

In [ ]:
demo_info =['pseudo_patient_key',
            'pre_dtm', 
            'pre_diff_hour', 
            'sex',
            'pre_age',
#             'age_group',
#             'district_board',
           'cls']
# select missingness less than 25% tets and HBA1C

tests_name = list(name_dict.values())
features = dataset.copy()[demo_info+tests_name]
missing = features.isnull().sum()
percent = features.isnull().sum() / features.isnull().count()
valid = features.notnull().sum()
missing_data = pd.concat([missing, valid,percent], axis=1, keys=["Missing","Valid", "Missing_percent"])
missing_data.sort_values("Missing_percent")

In [ ]:
# add to statistic result dataframe
stt["Missing Rate"] = missing_data.loc[indicators, "Missing_percent"].to_list()
# write to disk
stt.to_csv(os.path.join(OUT_PATH, "tables", "overall_statistics.csv"))

### 3.2 Exclude invalid features

In [ ]:
# exclude the features of missingness excessing 30% except for HBA1C
valid_tests = missing_data.loc[tests_name].query("Missing_percent < 0.3").index.to_list()
if not 'HBA1C' in valid_tests:
    valid_tests.append('HBA1C')

In [ ]:
# # Imputation
# df_train, df_test = train_test_split(ds, test_size=0.1, random_state=42)
# lr = LogisticRegression()
# numeric_col = tests_name + ["pre_age", "sex", "pre_diff_hour"]
# # imp = IterativeImputer(estimator=lr, missing_values=np.nan,max_iter=10, verbose=2, imputation_order='roman', random_state=0)
# imp = IterativeImputer(missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman', random_state=0)
# df_train[numeric_col] = imp.fit_transform(df_train[numeric_col])
# df_train[numeric_col] = df_train[numeric_col].clip(0)
# df_test[numeric_col] = imp.transform(df_test[numeric_col])
# df_test[numeric_col] = df_test[numeric_col].clip(0)
# df_train.isnull().sum()

## 4 Preprocessing
In this section, we will do:
1. drop null value records.
2. map sex to 0 and 1.
3. normalize the data and write the scaler to disk.
4. split the data

In [ ]:
# make a shallow copy so that we won't mess up with the original dataset
ds = dataset.copy()[demo_info + valid_tests]

### 4.1 Drop null value records

In [ ]:
#drop all the null value
ds = ds.dropna(how="any")

### 4.2 Map sex to 0 and 1.

In [ ]:
sex_mapper = {'F':0, 'M':1}
ds["sex"] = ds["sex"].apply(lambda x : sex_mapper[x])

### 4.3 Split the data
We split the data stratified on the class so the test set and training set has the same positive rate, we don't want to touch test set in the middle of anything about training.

In [ ]:
df_train, df_test = train_test_split(ds, test_size=0.1, random_state=42, stratify=ds["cls"])

### 4.4 Normalize the data and permenant the scaler

In [ ]:
# normalize the data using RobustScaler()
scaler = RobustScaler()
df_train[valid_tests] = scaler.fit_transform(df_train[valid_tests])
df_test[valid_tests] = scaler.transform(df_test[valid_tests])

# save the scaler
file = os.path.join(OUT_PATH, "models", "scaler.pkl")
pickle.dump(scaler, open(file, 'wb'))

In [ ]:
X_train = df_train[valid_tests + ["pre_age", "sex"]]
y_train = df_train["cls"]
X_test = df_test[valid_tests + ["pre_age", "sex"]]
y_test = df_test["cls"]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape
# write to result

## 5 Feature Selection
In this section we implement feature selection by **Logistic regression with L1 penalty**, since L1 penalty will generate sparse coefficient matrix, it can be used for feature selection by excluding the features with 0 coefficient.

we will do the following steps:
1. grid search for optimal parameter.
2. fit the model.
3. select features

### 5.1 Grid search for optimal parameter

In [ ]:
lr = LogisticRegression(penalty='l1', solver='liblinear')
grid = {"C": [0.001, 0.01, 0.1, 1, 10 ,100, 1000]}
search = GridSearchCV(estimator=lr, param_grid=grid)

In [ ]:
search.fit(X_train, y_train)
search.best_params_

### 5.2 Fit the model

In [ ]:
features = np.array(X_train.columns).reshape(1,-1)
features.shape

In [ ]:
gs_model = LogisticRegression(**search.best_params_, penalty='l1', solver='liblinear')
gs_model.fit(X_train, y_train)
coefficients = gs_model.coef_

### 5.3 Feature selection

In [ ]:
importance = np.abs(coefficients)
valid_features = features[importance > 0]
valid_features

In [ ]:
fs_conf = {}
fs_conf["C"] = search.best_params_["C"]
fs_conf["valid_features"] = valid_features
log["feature_selection"] = fs_conf

In [ ]:
X_train = X_train[valid_features]
X_test = X_test[valid_features]

In [ ]:
X_train.shape, X_test.shape

## 6. Resample the data
I use random sampling method to balance the data. To elaborate the training, I decided to do both the oversampling and undersampling for the traning set.

In [ ]:
ros = RandomOverSampler(random_state=0)
X_train_os, y_train_os = ros.fit_resample(X_train, y_train)
y_train_os.value_counts()

In [ ]:
uos = RandomUnderSampler(random_state=0)
X_train_us, y_train_us = uos.fit_resample(X_train, y_train)
y_train_us.value_counts()

## 7. Machine Learning Models

In this section, I will establish evaluating metrics for  the model performance and by which we compare each model performance. Specifically, the section includes following steps:

1. Set up metircs: precision, recall, f1-score, accuracy.
2. Train the models and display each metrics.
3. Discussion.

### 7.1 Evaluating Metrics

False negatives(fn), False positives(fn), True negatives(tn), True positives(tp)

Accuracy: the percentage of predicted positives that were correctly classified > true_samples / total_samples 

Recall: the percentage of **actual** positives that were correctly classified > tp / (tp + fn)

Precision: the percentage of **predicted** positives that were correctly classified > tp / (tp + fp) 



In [ ]:
def get_scores(model):
    
    y_pred_test = model.predict(X_test)
    
    print("=========================================================")
    print("Metrics for model " + model.__class__.__name__)
    report = classification_report(y_test, y_pred_test, target_names=['no incidence', 'incidence ocurred'])
    print(report)
    with open('out.txt', 'a') as f:
        print("=========================================================", file=f)
        print("Metrics for model " + model.__class__.__name__, file=f)
        print(report, file=f)
        print("\n\n", file=f)
#     plot confusion matrix
    cm = confusion_matrix(y_test, y_pred_test, labels=[0.0,1.0])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no incidence', 'incidence ocurred'])
    disp.plot(values_format='d')
    fig = disp.figure_
    fig.set_figwidth(8)
    fig.set_figheight(8)
    plt.grid(False)
    plt.title("Confusion Matrix for " + model.__class__.__name__, fontsize=12, fontweight='bold')
    plt.savefig("Confusion Matrix for " + model.__class__.__name__ + ".jpeg", bbox_inches='tight', dpi=400)

In [ ]:
def plot_auc(model):
    plt.figure(figsize=(5,5))
    y_test_score = model.decision_function(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_score)
    roc_auc = auc(fpr, tpr)
    plt.title("ROC")
    plt.plot(fpr, tpr, 'b', label='AUC = %0.4f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1], 'r--')
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.01])
    plt.title("ROC for model " + model.__class__.__name__)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

### 7.2 Training and evaluating

1. Logistic Regression(baseline)
2. Decision Tree
3. RandomForestClassifier
4. Ada Boost
5. GradientBoostingClassifier
6. VotingClassifier
7. SVM

#### 1.1) Logistic Regression(oversampling)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

logreg = LogisticRegression()

grid = {"C": [0.001, 0.01, 0.1, 1, 10 ,100, 1000], 'penalty':['l1', 'l2']}

logreg_cv = GridSearchCV(logreg, grid, cv=5, scoring='balanced_accuracy')

logreg_cv.fit(X_train_os, y_train_os)

print("Best parameters " , logreg_cv.best_params_)

logreg2 = LogisticRegression(**logreg_cv.best_params_)
logreg2.fit(X_train_os, y_train_os)
get_scores(logreg2)
plot_auc(logreg2)

#### 1.2) Logistic Regression(Undersampling)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

logreg = LogisticRegression()

grid = {"C": [0.001, 0.01, 0.1, 1, 10 ,100, 1000], 'penalty':['l1', 'l2']}

logreg_cv = GridSearchCV(logreg, grid, cv=5, scoring='balanced_accuracy')

logreg_cv.fit(X_train_us, y_train_us)

print("Best parameters " , logreg_cv.best_params_)

logreg2 = LogisticRegression(**logreg_cv.best_params_)
logreg2.fit(X_train_us, y_train_us)
get_scores(logreg2)
plot_auc(logreg2)

#### 2.1) Decision Tree (oversampling)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_os, y_train_os)
get_scores(dt)

#### 2.1) Decision Tree (undersampling)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_us, y_train_us)
get_scores(dt)

#### 3.1) RandomForestClassifier(oversampling)

In [ ]:
rfc = RandomForestClassifier(class_weight={0.0:1,1.0:3}, random_state=42)
rfc.fit(X_train_os, y_train_os)
get_scores(rfc)

#### 3.2) RandomForestClassifier(undersampling)

In [ ]:
rfc = RandomForestClassifier(class_weight={0.0:1,1.0:3}, random_state=42)
rfc.fit(X_train_us, y_train_us)
get_scores(rfc)

#### 4.1) AdaBoost(oversampling)

In [ ]:
adb = AdaBoostClassifier(n_estimators=100)
adb.fit(X_train_os, y_train_os)
get_scores(adb)
plot_auc(adb)

#### 4.2) AdaBoost(undersampling)

In [ ]:
adb = AdaBoostClassifier(n_estimators=100)
adb.fit(X_train_us, y_train_us)
get_scores(adb)
plot_auc(adb)

#### 5.1) Gradient Tree Boosting(oversmapling)

In [ ]:
xgb= GradientBoostingClassifier(n_estimators=100, max_depth=1, random_state=0).fit(X_train_os, y_train_os)
get_scores(xgb)
plot_auc(xgb)

#### 5.2) Gradient Tree Boosting(undersmapling)

In [ ]:
xgb= GradientBoostingClassifier(n_estimators=100, max_depth=1, random_state=0).fit(X_train_us, y_train_us)
get_scores(xgb)
plot_auc(xgb)

#### 6.1) VotingClassifier(oversampling)

In [ ]:
VC = VotingClassifier(
    estimators=[('xgb', xgb), ('adb', adb)],
    voting="soft",
    weights=[2,1],
    flatten_transform=True)
VC = VC.fit(X_train_os, y_train_os)
get_scores(VC)

#### 6.2) VotingClassifier(undersampling)

In [ ]:
VC = VotingClassifier(
    estimators=[('xgb', xgb), ('adb', adb)],
    voting="soft",
    weights=[2,1],
    flatten_transform=True)
VC = VC.fit(X_train_us, y_train_us)
get_scores(VC)

#### 7) SVM

In [ ]:
from sklearn.svm import SVC
SVM = SVC(gamma="auto").fit(X_train_us, y_train_us)
get_scores(SVM)
plot_auc(SVM)

## 6. Deep Learning Model

First, we need to split the data

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tempfile

In [ ]:
# early stopping callback on AUC
class AUCStopping(keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_auc') >= 0.81 and logs.get('val_recall') >= 0.9):
            print("\n Early stopping beacause validation auc excesses 80%")
            self.model.stop_training = True

callbacks: AUCStopping = AUCStopping()
    

Set up the metrics and define the model creating method.

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'), # we focus on recall metrics
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    # build the model, this model can be seen as a logistic regression but with extra drop-out layers 
    # for avoiding overfitting     
    model = keras.Sequential([
        keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[-1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.5), # avoid overfitting
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])
    
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=1e-3), #adam is not sensitive to different scale of loss
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    return model

In [ ]:
#hyper parameters
EPOCHS = 100
BATCH_SIZE = 2000 # make sure each batch containes positive case
total = y_test.count()
pos = y_test.sum()
neg = total - pos
weight_0 = (1 / neg) * total / 2
weight_1 = (1 / pos) * total / 2
weight_0, weight_1
CLASS_WEIGHT = {0:weight_0, 1:weight_1}

### 7.1 Baseline model

In [ ]:
# results = model.evaluate(X_train, y_train, batch_size=2000, verbose=0)
# print("loss: {:0.4f}".format(results[0]))

### 7.2 Train the model

In [ ]:
model = make_model()
baseline_history = model.fit(
    X_train,
    y_train.astype(np.int64),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks = [callbacks],
    verbose = 1
)

Assign weight to different classes so that the model will focus on the minority class.

In [ ]:
weighted_model = make_model()

class_weight = {0: 1, 1: 2}
weighted_history = weighted_model.fit(
    X_train_us,
    y_train_us.astype(np.int64),
    batch_size=BATCH_SIZE,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks = [callbacks],
    class_weight=class_weight
)

In [ ]:
y_test_pred = (weighted_model.predict(X_test) > 0.5).astype("int")
y_test_pred

In [ ]:
temp = y_test.to_numpy().reshape(-1,1)
(temp == y_test_pred).mean()

In [ ]:
tp = (temp * (y_test_pred == 1)).sum()
fn = (temp * (y_test_pred == 0)).sum()
recall = tp / (tp+fn)
recall

In [ ]:
weighted_model.save(os.path.join(OUT_PATH,"models", "weighted_model"))

In [ ]:
model_conf = {}
model_conf["epochs"] = EPOCHS
model_conf["BATCH_SIZE"] = BATCH_SIZE
model_conf["weight"] = class_weight
dl_info = {}
evaluation = weighted_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=2)
dl_info["model_conf"] = model_conf
dl_info["loss"] = evaluation[0]
dl_info["accuracy"] = evaluation[4]
dl_info["precision"] = evaluation[5]
dl_info["recall"] = evaluation[6]
dl_info["recall"] = evaluation[7]
log["dl_info"] = dl_info

In [ ]:
evaluation = weighted_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=2)
df_info["model_conf"] = model_conf
dl_info["loss"] = evaluation[0]
dl_info["accuracy"] = evaluation[4]
dl_info["precision"] = evaluation[5]
dl_info["recall"] = evaluation[6]
dl_info["recall"] = evaluation[7]
log["dl_info"] = dl_info

In [ ]:
with open(os.path.join(OUT_PATH, "result.json"), 'w', encoding='utf-8') as f:
    json.dump(log, f, ensure_ascii=False, indent=4)

### 7.3 Display Metrics

In this sub section, I will display the metrics I set up against the training epochs for both baseline model and weighted model. This will include:
1. Training Loss
2. Recall
3. Precision
4. Accuracy

And also I will investigate AUC for both models.

In [ ]:
def print_metrics(model):
    results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
    for name, value in zip(model.metrics_names, results):
        print(name, ': ', value)
    print()

In [ ]:
print_metrics(model)
print_metrics(weighted_model)

In [ ]:
import tensorflow as tf
from tensorflow import keras
y_pred = np.array([0.3 , 0.1, 0.1, 0.7])
y_true = np.array([1,0,1,0])

def recallLoss(y_true, y_pred):
    threshold = 0.2
    y_pred = (y_pred > threshold).astype(np.int32)
    recall = y_pred[y_true == 1].mean()
    return recall
recallLoss(y_true, y_pred)

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'), # we focus on recall metrics
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
]

In [ ]:
y_pred = np.array([0.3 , 0.1])
y_true = np.array([1,0])
y_pred = (y_pred > 0.2).astype(np.int32)
y_pred[y_true == 1].mean()

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
mpl.rcParams['figure.figsize'] = (12,10)
def plot_metrics(history):
    metrics = ['loss', 'accuracy', "recall", "precision"]
    for n, metric in enumerate(metrics):
        name = metric.replace("_", " ").capitalize()
        plt.subplot(2, 2, n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label="Train")
        plt.plot(history.epoch, history.history['val_'+metric], color=colors[0], linestyle="--", label="Val")
        plt.xlabel("Epoch")
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])
        plt.legend()
    plt.savefig(os.path.join(OUT_PATH, "charts", "training_history.png"))

In [ ]:
plot_metrics(weighted_history)

In [ ]:
# train_predictions_baseline = model.predict(X_train, batch_size=BATCH_SIZE)
# test_predictions_baseline = model.predict(X_test, batch_size=BATCH_SIZE)
train_predictions_weighted = weighted_model.predict(X_train, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(X_test, batch_size=BATCH_SIZE)

In [ ]:
new_model = tf.keras.models.load_model(os.path.join(OUT_PATH, "models", "weighted_model"))
new_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=2)

In [ ]:
train_predictions_weighted

In [ ]:
#plot ROC
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)
    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([0,100])
    plt.ylim([0,100])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
# plot_roc("Train Baseline", y_train, train_predictions_baseline, color=colors[0])
# plot_roc("Test Baseline", y_test, test_predictions_baseline, color=colors[0], linestyle='--')
plot_roc("Train Weighted", y_train, train_predictions_weighted, color=colors[0], linestyle='--')
plot_roc("Test Weighted", y_test, test_predictions_weighted, color=colors[1], linestyle='--')
plt.legend(loc='lower right')
plt.savefig(os.path.join(OUT_PATH, "charts", "Model ROC"))

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred = weighted_model.predict(X_test).ravel()
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
AUC = auc(fpr, tpr)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
plt.plot([0,1],[0,1], 'k--')
plt.plot(fpr, tpr, label="weighted model (area = {:.3f})".format(AUC))
plt.xlabel("True Positive Rate")
plt.ylabel("False Postive Rate")
plt.title("ROC Curve")
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5,5))
fig.axes

In [ ]:
# plot confusion matrix
def plot_cm(y, predictions, threshold=0.5):
    cm = confusion_matrix(y, predictions > threshold)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no incidence', 'incidence ocurred'])
    disp.plot(ax=ax)
    plt.title('Confusion matrix @{:.2f} threshold'.format(threshold))
    plt.ylabel('True label')
    plt.xlabel("Predicted label")
    plt.savefig("./foo.png")
plot_cm(y_test, weighted_model.predict(X_test))

In [ ]:
weighted_model.evaluate(X_test, y_test, verbose=1)

## 8.Feature permutation importance

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
from typing import Callable
y_test_pred = weighted_model.predict(X_test)
roc_auc_score(y_test, y_test_pred, average="weighted")
valid_features

In [ ]:

X_test["creatinine"].sample(frac=1 ,replace=False, random_state=42)

In [ ]:
def permutation_importance(model: Callable, X: pd.DataFrame, y: pd.Series, n_repeats: int, metric_fn: Callable, **params) -> dict:
    '''
    the method return the mean score and std through out the n_repeat, the score is computed by the provided callable type
    metric
    
    Assumption & Method
    It assumes that the higher the score, the better the performance of the model is, and we use difference to measure the model
    reliance of the feature.
    
    Arg:
        model: model interface implements predicit method that return the decision score of the prediciton 
        X: input of validation set to be permutated
        y: ground truth
        n_repeats: the times of the iteration
        metric_fn: the callable type to compute the score
        params: addtional parameter to pass into the metric_funtion
    
    Return:
        mean score over n repeats for each features.
    '''
    features = list(X.columns)
    y_pred = model.predict(X)
    original_score = metric_fn(y, y_pred, **params)
    imp = {}
    for _ in range(n_repeats):
        for f in features:
            X_new = X.copy() #copy the reference to the original dataframe
            X_new[f] = X[f].sample(frac=1, replace=False).to_list()
            y_pred = model.predict(X_new)
            score = metric_fn(y, y_pred, **params)
            diff = original_score - score
            # assert the new score should not be greater than before
            prev = imp.get(f, 0)
            # add the new score difference
            imp[f] = prev+ diff
    for f in imp:
        imp[f] = imp[f] / n_repeats
    return imp

In [ ]:
permutation_importance(weighted_model, X_test, y_test, 30, roc_auc_score, average="weighted")

# Time-dependent Model

In [ ]:
name_dict = {
    5200279: "creatinineRenal",
    5200289: "cholesHDL",
    5200290: "choles",
    5200295: "creatinine",
    5200305: "glucose",
    5200306: "fastingGlucose",
    5200325: "triglyceride",
    5200406: "cholesLDL_1",
    5201215: "glucoseInBlood",
    5203289: "proteinCreatinineRatio" ,   
    5200345: "albumin",
    5200346: "albumin24h",
    5200387: "potassiumSerumOrPlasma",
    5200393: "proteinUrine",
    5200394: "proteinUrine24h",
    5200402: "albuminCreatinineRatio",
    5200485: "HBA1C",
    5200547: "albuminUnspecifiedTime",
    5200679: "cholesLDL_2",
    5200715: "creatinineRenalClearance",
    5200935: "microalbuminCreatinineRatio",
    5201051: "proteinCreatinineMassRatio",
    5204348: "glomerularFiltrationRate"
}
test = list(name_dict.values())

tests_dict = {5200289: "cholesHDL",
 5200290: "choles",
 5200295: "creatinine",
 5200306: "fastingGlucose",
 5200325: "triglyceride",
 5200485: "HBA1C"
}
tests_id = list(tests_dict.keys())
tests_name = list(tests_dict.values())
demo_info =['pseudo_patient_key',
            'pre_dtm', 
            'pre_diff_hour', 
            'sex',
            'pre_age']

In [ ]:
# patients data
patients = pd.read_csv(r'../tables/output/group_patient_age.csv', index_col=0)

# define the file path and tables path for file reading
file_path = r'../DATAFILE'
tid_to_eid_path = r'iams_entity_concept'
labresult_cps_path = 'lis_cps_result_data'
labresult_hms_path = 'lis_hms_result_data'
 
# read the fragment files and concat them
usecols = ["pseudo_patient_key", "reference_dtm", "diff_in_hour_reference_dtm", "result_str", "entity_id", "si_unit", "si_numeric"]
labresult_cps = tools.fileReader(file_path, labresult_cps_path, usecols=usecols)
labresult_hms = tools.fileReader(file_path, labresult_hms_path, usecols=usecols)
tid_to_eid = tools.fileReader(file_path, tid_to_eid_path)

# the datafield of cps and hms are the same, so we can concate them.
labresult = pd.concat([labresult_cps, labresult_hms])

# patients data
patients = pd.read_csv(r'../tables/output/group_patient_age.csv', index_col=0)

# define the file path and tables path for file reading
file_path = r'../DATAFILE'
tid_to_eid_path = r'iams_entity_concept'
labresult_cps_path = 'lis_cps_result_data'
labresult_hms_path = 'lis_hms_result_data'
 
# read the fragment files and concat them
usecols = ["pseudo_patient_key", "reference_dtm", "diff_in_hour_reference_dtm", "result_str", "entity_id", "si_unit", "si_numeric"]
labresult_cps = tools.fileReader(file_path, labresult_cps_path, usecols=usecols)
labresult_hms = tools.fileReader(file_path, labresult_hms_path, usecols=usecols)
tid_to_eid = tools.fileReader(file_path, tid_to_eid_path)
del labresult_cps
del labresult_hms

# the datafield of cps and hms are the same, so we can concate them.
labresult = pd.concat([labresult_cps, labresult_hms])

patients = patients.query("label != 2")
patients = patients.query("diab_age >= 18.0 or diab_age.isnull()", engine="python")

eid = tid_to_eid[tid_to_eid.term_id.isin(tests_id)]["entity_id"]

features=["pseudo_patient_key", "age", 'test_name', 'si_numeric']

# left join the table with the patients test
dataset = pd.merge(left=patients, right=labresult[labresult.entity_id.isin(eid)], how='inner', on="pseudo_patient_key")

f = lambda x : int(x[:4])
dataset = dataset.assign(age=dataset["reference_dtm"].apply(f) - dataset["dob_Y"].apply(f))
# merge with tid
dataset = pd.merge(left=dataset, right=tid_to_eid, how='inner', on="entity_id")
# map the name of the tests
dataset["test_name"] = dataset["term_id"].apply(lambda x : tests_dict[x])
# truncate the dataset at the moment of the prediabetes
dataset = dataset.query("reference_dtm <= pre_dtm")
# exclude the patient later than 2016-12-31
dataset = dataset.query("pre_dtm <= '2016-12-31'")

# feature columns
f_col = dataset[features]
f_col = f_col.replace(r'""', np.nan) # mark "" as null value
f_col["si_numeric"] = f_col["si_numeric"].astype('float')
# group by patient and age
test_mean = f_col.groupby(["pseudo_patient_key","age", "test_name"], as_index=False).mean()

test_mean.sort_values(["pseudo_patient_key", "test_name", "age"])

# map the class for each patient
right = dataset[["pseudo_patient_key","label"]].drop_duplicates()
ds = pd.merge(left=test_mean, right=right, on="pseudo_patient_key")
# encode the class
test_label_dict = {"cholesHDL":0,
"choles":1,
"creatinine":2,
"fastingGlucose":3,
"triglyceride":4,
 "HBA1C":5
}
ds["test_label"] = ds["test_name"].apply(lambda x : test_label_dict[x])
# drop null value
ds = ds.dropna(how="any")

# find the max length of test sequence
max_len = ds.groupby("pseudo_patient_key")["pseudo_patient_key"].count().max()
n_patient = ds.pseudo_patient_key.nunique()
ds = ds.sort_values(["pseudo_patient_key","age", "test_name"])


patient_id = ds["pseudo_patient_key"].unique()
head = 0
tail = 0
n = ds.shape[0]
def to_X(df):
    n = df.shape[0]
    max_len = df.groupby("pseudo_patient_key")["pseudo_patient_key"].count().max()
    n_patient = df.pseudo_patient_key.nunique()
    df = df.sort_values(["pseudo_patient_key","age", "test_name"])
    patient_id = df["pseudo_patient_key"].unique()
    X = np.zeros((n_patient, max_len, 3))
    head = 0
    tail = 0
    y = np.zeros((n_patient,))
    for i, id in enumerate(patient_id):
        y[i] = df.iloc[tail]["label"]
        while(df.iloc[tail]["pseudo_patient_key"] == id):
            tail += 1
            if tail == n:
                break
        diff = tail - head
        X[i,:diff, :] = df.iloc[head:tail][["test_label", "age", "si_numeric"]]
        head = tail
        if i % 100 == 0:
            print(f"finished {i}/{n_patient}")
    return X, y
X , y = to_X(ds)   
# split the data to test and train set
# train_df, test_df = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'), # we focus on recall metrics
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
    
]

In [ ]:
# build the model
EMB_DIM = 3 # the dimension of defining the state
LSTM_UNITS = 128
CLASS_NUM = 4
MAX_LEN = 86
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='causal', input_shape=[MAX_LEN,EMB_DIM]),
    tf.keras.layers.LSTM(LSTM_UNITS),
    tf.keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5), # avoid overfitting
    tf.keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5), # avoid overfitting
    tf.keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.5), # avoid overfitting
    tf.keras.layers.Dense(1, activation='sigmoid')
])

def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

model_lstm.compile(loss=keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=METRICS)
model_lstm.summary()

In [ ]:
EPOCHS = 100
BTACH_SIZE = 10
weight_minor = 1
weight_major = 1000
class_weight = {0: weight_major, 1: weight_minor}
weighted_history = model_lstm.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data = (X_test, y_test),
    class_weight=class_weight,
    verbose=1
)

## Survival Model

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
demo_info.remove("pre_age")

In [ ]:
cox_ds = dataset[valid_features.tolist() + demo_info + ["diab_dtm","label", "diab_diff_hour"]]
pre_dtm = pd.to_datetime(cox_ds["pre_dtm"])
cox_ds["prog_pd_hour"] = (cox_ds["diab_diff_hour"] - cox_ds["pre_diff_hour"])
#fill the na value with the hour difference between pre and 2019-12-16
# we use 2019-12-16 as censored date to compromise the bias of date time set by HA
cox_ds["prog_pd_hour"] = cox_ds["prog_pd_hour"].fillna((pd.to_datetime("2019-12-16") - pre_dtm).dt.days * 24)
# mapping sex {"F": 0, "M":1}
sex_mapper = {"F": 0, "M":1}
cox_ds["sex"] = cox_ds["sex"].apply(lambda x : sex_mapper[x])

In [ ]:
cox_train = cox_ds[valid_features.tolist() + ["sex", "prog_pd_hour", "label"]]
cox_train = cox_train.dropna(how='any')

In [ ]:
cox_train.groupby("label").mean()

In [ ]:
scaler = RobustScaler()
cox_train[valid_features.tolist()] = scaler.fit_transform(cox_train[tests_name])

In [ ]:
cph = CoxPHFitter()
cph.fit(cox_train, duration_col='prog_pd_hour', event_col='label')
cph.print_summary()